In [1]:
# Import libraries
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from pytorch_tabnet.tab_model import TabNetClassifier
import pandas as pd
import numpy as np
import hyperopt
from hyperopt import fmin, tpe, Trials, hp
import hyperopt.pyll.stochastic
from hyperopt import pyll

# Hyperopt function
def search_reg(space, max_eval, output_path):
    T = Trials()
    best = fmin(optimiser, space, algo=tpe.suggest, max_evals=max_eval, trials=T, pass_expr_memo_ctrl=True)
    
    logs = []
    results = []
    
    T.trials
    for i in T.trials:
        logs.append(i['misc']['vals'])
        results.append(i['result']['loss'])
    
    trial_data = pd.DataFrame(logs)
    trial_data['F1'] = results
    trial_data.to_csv(output_path)
    
    return hyperopt.space_eval(space, best)

In [ ]:
x = pd.read_csv('../../Data/Missing_0_Mask/X_train_0.csv').to_numpy()
y = pd.read_csv('../../Data/Y_train.csv').to_numpy().reshape(-1)
xt = pd.read_csv('../../Data/Missing_0_Mask/X_test_0.csv').to_numpy()
yt = pd.read_csv('../../Data/Y_test.csv').to_numpy().reshape(-1)

In [3]:
acc = np.zeros(100)
i = 0

In [4]:
def Kfold(data, labels,i, n_d, n_a, n_steps, lr, gamma, n_independent, n_shared, momentum, best = None):
    
    kf = StratifiedKFold(n_splits = 5, shuffle = True)
    kf.get_n_splits(data, labels)
    
    VS_arr = []
    Acc_arr = []
    for fold, (train_index, test_index) in enumerate(kf.split(data, labels)):
        print('--------------------------------')
        print(f'FOLD {fold}')
        print('--------------------------------')
        
        x_train, x_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        clf = TabNetClassifier(n_d =n_d, n_a = n_d, n_steps=n_steps,  cat_idxs = [24,25], optimizer_params=dict(lr=lr), gamma=1+gamma, n_independent=n_independent, n_shared = n_shared, momentum= momentum)
        clf.fit(x_train,y_train, eval_set=[(x_test,y_test),(xt, yt)], weights = 1, max_epochs = 5)  
        
        y_pred=clf.predict(xt)
        f1 = f1_score(y_true=yt, y_pred=y_pred,average='macro')
        
        Acc_arr.append(np.mean(yt == y_pred))
        
        VS_arr.append(f1)
        
        print(f'Fold {fold} Validation Score: {f1}')
        
        # Early Stopping
        if best != None:
            if f1 < best:
                print('********************************')
                print('F1 Score too low ... stopped early.')
                print('********************************')
                return f1
        
    acc[i] = min(Acc_arr)
    i += 1
    
    final_F1 = min(VS_arr)
    print('--------------------------------')
    print(f'\n\nMinimum Validation Score: {final_F1}')
    print('--------------------------------')
    
    return final_F1

In [5]:
space = {'n_d':hp.randint('n_d',56),
         'n_steps':hp.randint('n_steps',7),
         'lr':hp.choice('lr', [1e-2,1e-3,1e-4,1e-5,1e-6]),
         'gamma':hp.uniform('gamma',0,1),
         'n_independent':hp.randint('n_independent',4),
         'n_shared':hp.randint('n_shared',4),
         'momentum':hp.uniform('momentum',0.01,0.4),
        }

In [6]:
def optimiser(expr, memo, ctrl):
    try:
        config = pyll.rec_eval(expr, memo=memo)
        print(config)
        if len(ctrl.trials.losses()) > 1:
            best_score = -min(value for value in ctrl.trials.losses() if value is not None)
        else:
            best_score = -5

        return -Kfold(x, y,i, n_d = 8+config['n_d'], n_a = config['n_d'], n_steps=3+config['n_steps'], lr=config['lr'], gamma=1+config['gamma'], n_independent=1+config['n_independent'], n_shared =1+config['n_shared'], momentum=config['momentum'], best = best_score)
    
    except:
        return 99

In [7]:
search_reg(space, 100, '1_TabNet.csv')

{'gamma': 0.03317348406395182, 'lr': 0.0001, 'momentum': 0.3469393397858053, 'n_d': 4, 'n_independent': 2, 'n_shared': 1, 'n_steps': 4}
--------------------------------                     
FOLD 0                                               
--------------------------------                     
Device used : cuda                                   
epoch 0  | loss: 4.16309 | val_0_accuracy: 0.04458 | val_1_accuracy: 0.05253 |  0:01:01s
epoch 1  | loss: 3.22484 | val_0_accuracy: 0.06384 | val_1_accuracy: 0.06903 |  0:02:02s
epoch 2  | loss: 2.81366 | val_0_accuracy: 0.12243 | val_1_accuracy: 0.09789 |  0:03:03s
epoch 3  | loss: 2.58407 | val_0_accuracy: 0.15173 | val_1_accuracy: 0.10111 |  0:04:04s
epoch 4  | loss: 2.41225 | val_0_accuracy: 0.12405 | val_1_accuracy: 0.10424 |  0:05:05s
Stop training because you reached max_epochs = 5 with best_epoch = 4 and best_val_1_accuracy = 0.10424
Best weights from best epoch are automatically used! 
Fold 0 Validation Score: 0.04517554588350033  

--------------------------------                                                     
FOLD 0                                                                               
--------------------------------                                                     
Device used : cuda                                                                   
epoch 0  | loss: 1.11157 | val_0_accuracy: 0.44362 | val_1_accuracy: 0.34264 |  0:01:18s
epoch 1  | loss: 0.66155 | val_0_accuracy: 0.58257 | val_1_accuracy: 0.49507 |  0:02:36s
epoch 2  | loss: 0.68454 | val_0_accuracy: 0.61354 | val_1_accuracy: 0.55096 |  0:03:54s
epoch 3  | loss: 0.53511 | val_0_accuracy: 0.65946 | val_1_accuracy: 0.51946 |  0:05:13s
epoch 4  | loss: 0.45037 | val_0_accuracy: 0.69148 | val_1_accuracy: 0.54989 |  0:06:31s
Stop training because you reached max_epochs = 5 with best_epoch = 2 and best_val_1_accuracy = 0.55096
Best weights from best epoch are automatically used!                                   
Fold 0 Validation Sc

Best weights from best epoch are automatically used!                                   
Fold 0 Validation Score: 0.3614855088086712                                            
--------------------------------                                                       
FOLD 1                                                                                 
--------------------------------                                                       
Device used : cuda                                                                     
epoch 0  | loss: 1.03936 | val_0_accuracy: 0.47435 | val_1_accuracy: 0.41807 |  0:00:33s
epoch 1  | loss: 0.64805 | val_0_accuracy: 0.58624 | val_1_accuracy: 0.53432 |  0:01:06s
epoch 2  | loss: 0.53201 | val_0_accuracy: 0.63027 | val_1_accuracy: 0.58926 |  0:01:39s
epoch 3  | loss: 0.46999 | val_0_accuracy: 0.68273 | val_1_accuracy: 0.57693 |  0:02:12s
epoch 4  | loss: 0.4249  | val_0_accuracy: 0.73641 | val_1_accuracy: 0.60608 |  0:02:45s
Stop training because you r

Stop training because you reached max_epochs = 5 with best_epoch = 4 and best_val_1_accuracy = 0.62272
Best weights from best epoch are automatically used!                                  
Fold 4 Validation Score: 0.32871067276788896                                          
--------------------------------                                                      
                                                                                      

Minimum Validation Score: 0.32871067276788896
--------------------------------                                                      
{'gamma': 0.9740241374688179, 'lr': 1e-05, 'momentum': 0.18714873808803692, 'n_d': 33, 'n_independent': 2, 'n_shared': 0, 'n_steps': 4}
--------------------------------                                                      
FOLD 0                                                                                
--------------------------------                                                      
Device used : cuda

Fold 0 Validation Score: 0.2029254885833198                                            
********************************                                                       
F1 Score too low ... stopped early.                                                    
********************************                                                       
{'gamma': 0.38016126391860905, 'lr': 1e-05, 'momentum': 0.10151456036061067, 'n_d': 1, 'n_independent': 3, 'n_shared': 2, 'n_steps': 6}
--------------------------------                                                      
FOLD 0                                                                                
--------------------------------                                                      
Device used : cuda                                                                    
epoch 0  | loss: 4.62316 | val_0_accuracy: 0.12634 | val_1_accuracy: 0.12711 |  0:01:35s
epoch 1  | loss: 4.55444 | val_0_accuracy: 0.1831  | val_1_accuracy: 0.1710

...

Best weights from best epoch are automatically used!                                  
Fold 0 Validation Score: 0.3299200867537872                                           
********************************                                                      
F1 Score too low ... stopped early.                                                   
********************************                                                      
{'gamma': 0.6673867052791098, 'lr': 0.01, 'momentum': 0.3158789136888592, 'n_d': 26, 'n_independent': 1, 'n_shared': 3, 'n_steps': 1}
--------------------------------                                                      
FOLD 0                                                                                
--------------------------------                                                      
Device used : cuda                                                                    
epoch 0  | loss: 0.9122  | val_0_accuracy: 0.54722 | val_1_accuracy: 0.51796 |  0:0

********************************                                                      
{'gamma': 0.4859591649604822, 'lr': 1e-05, 'momentum': 0.3083919163603426, 'n_d': 11, 'n_independent': 1, 'n_shared': 1, 'n_steps': 0}
--------------------------------                                                      
FOLD 0                                                                                
--------------------------------                                                      
Device used : cuda                                                                    
epoch 0  | loss: 3.39291 | val_0_accuracy: 0.13301 | val_1_accuracy: 0.22741 |  0:00:27s
epoch 1  | loss: 3.2493  | val_0_accuracy: 0.12917 | val_1_accuracy: 0.27837 |  0:00:55s
epoch 2  | loss: 3.11693 | val_0_accuracy: 0.11211 | val_1_accuracy: 0.20958 |  0:01:22s
epoch 3  | loss: 3.01238 | val_0_accuracy: 0.12387 | val_1_accuracy: 0.19971 |  0:01:50s
epoch 4  | loss: 2.91188 | val_0_accuracy: 0.1054  | val_1_accuracy: 0.185

Device used : cuda                                                                     
epoch 0  | loss: 1.31269 | val_0_accuracy: 0.4402  | val_1_accuracy: 0.39714 |  0:01:15s
epoch 1  | loss: 0.76284 | val_0_accuracy: 0.64537 | val_1_accuracy: 0.58241 |  0:02:32s
epoch 2  | loss: 0.58115 | val_0_accuracy: 0.58594 | val_1_accuracy: 0.53685 |  0:03:47s
epoch 3  | loss: 0.52179 | val_0_accuracy: 0.70111 | val_1_accuracy: 0.57453 |  0:05:03s
epoch 4  | loss: 0.48566 | val_0_accuracy: 0.64035 | val_1_accuracy: 0.54205 |  0:06:19s
Stop training because you reached max_epochs = 5 with best_epoch = 1 and best_val_1_accuracy = 0.58241
Best weights from best epoch are automatically used!                                   
Fold 0 Validation Score: 0.3209352948675444                                            
********************************                                                       
F1 Score too low ... stopped early.                                                    
************

Best weights from best epoch are automatically used!                                 
Fold 0 Validation Score: 0.14023975140246395                                         
********************************                                                     
F1 Score too low ... stopped early.                                                  
********************************                                                     
{'gamma': 0.4364391604986681, 'lr': 0.01, 'momentum': 0.289691583629053, 'n_d': 27, 'n_independent': 2, 'n_shared': 3, 'n_steps': 3}
--------------------------------                                                     
FOLD 0                                                                               
--------------------------------                                                     
Device used : cuda                                                                   
epoch 0  | loss: 1.06323 | val_0_accuracy: 0.45396 | val_1_accuracy: 0.44131 |  0:01:10s
epoc

{'gamma': 0.5448238063032632,
 'lr': 0.01,
 'momentum': 0.261249838793264,
 'n_d': 8,
 'n_independent': 1,
 'n_shared': 3,
 'n_steps': 1}

In [8]:
np.savetxt('acc.csv', acc)